In [ ]:
# default_exp web_scraping

# web scraping

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [ ]:
#export

#Webscrapping
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import datetime

#Save data
import re
import pandas as pd

#Resources
import random
from fastcore.script import * # @Callparser
from swfd.resources import back, getInfo


def getHTMLsfu(): Funcion que  por medio del driver de selenium hace el log en la EESSA y obtiene el código html conla información

In [ ]:
#export
def getHTMLsfu(currentdate,originaldate):

    source_code=""
    try:
        rutadriver=back()
        rutadriver=str(rutadriver)+"geckodriver"
        options = Options()
        options.add_argument('-headless')
        driver = webdriver.Firefox(executable_path=str(rutadriver), options=options)
        currentday = datetime.datetime.now()
        driver.get("https://esc.cbk.waw.pl/products/api.php?parameter=f10_7&start_date="+str(originaldate.year)+"%2F"+str(originaldate.month)+"%2F"+str(originaldate.day)+"+00%3A00&end_date="+str(currentdate.year)+"%2F"+str(currentdate.month)+"%2F"+str(currentdate.day)+"+00%3A00&output_type=html")
        sleep(random.uniform(8,10))

        
        user = driver.find_element_by_name("callback_0")
        password = driver.find_element_by_name("callback_1")
        button = driver.find_element_by_name("callback_2")

        credentials=getCredentials()
        
        if(credentials[0] or credentials[1]):
            user.send_keys(credentials[0])
            password.send_keys(credentials[1])
            button.click()
            sleep(random.uniform(15,10))

            elem = driver.find_element_by_xpath("//*")
            source_code = elem.get_attribute("outerHTML")
        else:
            print("user or password not found in txt")

    except: 
        e = sys.exc_info()[0]
        print("Exception: ", e)
        print("Make sure you have the internet and the selenium driver is in the sfw.")
    finally:
        driver.quit()
    
    return source_code




getCredentials(): Funcion que obtiene el user y password previamente escrito en el fichero infouser.txt para el login de la EESA

In [ ]:
#export
def getCredentials():
    credentials=[]
    user=getInfo("user")
    password=getInfo("password")
    credentials.append(user)
    credentials.append(password)
    return credentials


In [ ]:
#hide
#If txt not exist return [[], []]
getCredentials()

'getCredentials()'

getDatosSfu(html): Función que analiza y obtiene los datos (fecha y valor de sfu) del html

In [ ]:
#export
def getListDataSfu(html):

    datalist=[]
    
    oldsfu=re.findall('<tbody>(.*?)<td>1996-02-14',str(html),re.DOTALL)
    dailysfu=re.findall('<tr>(.*?)</tr>',str(oldsfu),re.DOTALL)
    for day in dailysfu:
        if(re.findall("(.*?)00:00:00</td>",day)):
            datedata=re.findall("<td>(.*?) 00:00:00</td>",day)
            sfudata=re.findall("</td><td>(.*?)<",day)
            datalist.append([datedata[0],sfudata[0]])
    
    newsfu=re.findall('<td>1996-02-13 00:00:00</td><td>68</td></tr>(.*?)</tbody></table></div></body></html>',str(html),re.DOTALL)
    dailysfu=re.findall('<tr>(.*?)</tr>',str(newsfu),re.DOTALL)
    for day in dailysfu:
        if(re.findall("(.*?)20:00:00</td>",day)):
            datedata=re.findall("<td>(.*?) 20:00:00</td>",day)
            sfudata=re.findall("</td><td>(.*?)<",day)
            datalist.append([datedata[0],sfudata[0]])           

    return datalist[::-1]

emptydata(datosSfu)

In [ ]:
#export
def emptydata(datosSfu):
    alldatasfu=[]
    #Dia actual porque el primer registro siempre sera hoy-1
    csvdaybefore=datetime.datetime.combine(datetime.datetime.today(), datetime.time.min)
    #csvdaybefore=csvdaybefore+datetime.timedelta(days=1)
    for record in datosSfu:
        realdaybefore=csvdaybefore-datetime.timedelta(days=1)
        csvdaybefore = datetime.datetime.strptime(record[0], '%Y-%m-%d')
        #Si el dia de ayer no es igual al de hoy-1 faltan datos de un dia
        if not(realdaybefore == csvdaybefore):
            while(not(realdaybefore == csvdaybefore)):
                alldatasfu.append([realdaybefore.strftime('%Y-%m-%d'),"-1"])
                realdaybefore=realdaybefore-datetime.timedelta(days=1)
                
                
        alldatasfu.append(record)
    return alldatasfu  

sfuScv(listaDatos): Función que guarda los datos(fecha y valor de sfu) en un fichero csv

In [ ]:
#export
def sfuScv(datalist):
    path=getInfo("csvdirectory")+"sfuData.csv"
    df = pd.DataFrame(datalist, columns= ['Date', 'Sfu'])
    df.to_csv(path, header = False, index = False)

In [ ]:
"""#hide
sfuScv([[1,2],[2,3],[3,4]])
path=getInfo("csvdirectory")+"sfuData.csv"
df = pd.read_csv(path,header=None)
df"""

,0,1
0,1,2
1,2,3
2,3,4


web_scraping()

In [ ]:
#export
def web_scraping():
    datosSfu=[]
    
    currentdate = datetime.datetime.now()
    originaldate=datetime.datetime(1949, 1, 1)
    sourcecode=getHTMLsfu(currentdate,originaldate)
    
    if(sourcecode!=""):
        datosSfu=getListDataSfu(sourcecode)
        datosSfu=emptydata(datosSfu)

        sfuScv(datosSfu)
    else:
        print("error")



getMaxValueSfu(): Saca el mayor valor de sfu historicamente

In [ ]:
#export
@call_parse
def main():
    web_scraping()


In [ ]:
#main()